# Setup for Google Colab

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 6. Data Preparation (7 Points):

In [18]:
#Importing all relevant packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from matplotlib import rcParams
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
import pickle
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
import plotly.offline as pyo

# 6.1 Load the dataset and display the dataframe (2 Points).

In [19]:
#Reading in Data Set and displaying first 5 rows
# OSI_g2 = pd.read_csv("../datasets/online_shoppers_intention.csv")

# for google colab
path = '/content/drive/MyDrive/Colab Notebooks/hw3/online_shoppers_intention.csv'
OSI_g2 = pd.read_csv(path)

# conver to int
OSI_g2["Weekend"] = OSI_g2["Weekend"].astype(int)
OSI_g2["Revenue"] = OSI_g2["Revenue"].astype(int)

# create numeric mapping for visitor type, drop original col
OSI_g2.insert(loc=16, column="VisitorTypeNumeric", value=pd.factorize(OSI_g2['VisitorType'])[0] + 1)
OSI_g2 = OSI_g2.drop('VisitorType', axis=1)

# rename cols
new_col_names_g2 = {
    'Administrative_Duration' : 'AdministrativeDuration',
    'Informational_Duration' : 'InformationalDuration',
    'ProductRelated_Duration' : 'ProductRelatedDuration'
}
OSI_g2.rename(columns=new_col_names_g2, inplace=True)

# create numeric mapping for month
month_numeric_encoding_g2 = {
    "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4,
    "May": 5, "Jun": 6, "Jul": 7, "Aug": 8,
    "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
}
OSI_g2["Month"] = OSI_g2["Month"].map(month_numeric_encoding_g2)

# sanity check
OSI_g2.head(6)

,Administrative,AdministrativeDuration,Informational,InformationalDuration,ProductRelated,ProductRelatedDuration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorTypeNumeric,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.0,0.0,2.0,1,1,1,1,1,0,0
1,0,0.0,0,0.0,2,64.000000,0.000000,0.100000,0.0,0.0,2.0,2,2,1,2,1,0,0
2,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.0,0.0,2.0,4,1,9,3,1,0,0
3,0,0.0,0,0.0,2,2.666667,0.050000,0.140000,0.0,0.0,2.0,3,2,2,4,1,0,0
4,0,0.0,0,0.0,10,627.500000,0.020000,0.050000,0.0,0.0,2.0,3,3,1,4,1,1,0
5,0,0.0,0,0.0,19,154.216667,0.015789,0.024561,0.0,0.0,2.0,2,2,1,3,1,0,0


# 6.2 Use describe to provide statistics on the pandas Dataframe (2 Points).

In [20]:
OSI_g2.describe()

,Administrative,AdministrativeDuration,Informational,InformationalDuration,ProductRelated,ProductRelatedDuration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorTypeNumeric,Weekend,Revenue
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12042.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,7.691496,2.124006,2.357097,3.147364,4.069586,1.151176,0.232603,0.154745
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,3.423429,0.911325,1.717277,2.401591,4.025169,0.376989,0.422509,0.361676
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,5.000000,2.000000,2.000000,1.000000,2.000000,1.000000,0.000000,0.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,8.000000,2.000000,2.000000,3.000000,2.000000,1.000000,0.000000,0.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,11.000000,3.000000,2.000000,4.000000,4.000000,1.000000,0.000000,0.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,12.000000,8.000000,13.000000,9.000000,20.000000,3.000000,1.000000,1.000000


# 6.3 Split the dataset into a Training set and a Test set. Justify your preferred split (3 Points).

In [21]:
# check for missing values
print(OSI_g2.isna().sum())

Administrative              0
AdministrativeDuration      0
Informational               0
InformationalDuration       0
ProductRelated              0
ProductRelatedDuration      0
BounceRates                 0
ExitRates                   0
PageValues                  0
SpecialDay                  0
Month                     288
OperatingSystems            0
Browser                     0
Region                      0
TrafficType                 0
VisitorTypeNumeric          0
Weekend                     0
Revenue                     0
dtype: int64


In [22]:
# impute missing values before proceeding with train/test
imputer = SimpleImputer(strategy='most_frequent')
OSI_imputed_g2 = pd.DataFrame(imputer.fit_transform(OSI_g2), columns=OSI_g2.columns)

# sanity check
print(OSI_imputed_g2['Revenue'].value_counts())

0.0    10422
1.0     1908
Name: Revenue, dtype: int64


In [23]:
# set up train-test split
X_g2 = OSI_imputed_g2.loc[:, OSI_imputed_g2.columns != 'Revenue'].to_numpy()
y_g2 = OSI_imputed_g2.iloc[:,-1:].to_numpy()

X_train_g2, X_test_g2, y_train_g2, y_test_g2 = train_test_split(X_g2, y_g2, test_size=0.30, random_state=1859)

While the dataset is pretty large (over 12,000 rows), the dataset is also very imbalanced. As such, instead of going with the typical 80-20 train-test split, we'll go with a 70-30 split to ensure that there's enough data to evaluate the performance of the model on the minority class.

# 7. Classification Routine (12 Points):
Execute a classification routine using RandomForestClassifier(), BaggingClassifier(), and XGboostclassifier(). Independently output the accuracy box plot as discussed in class. Use any package you are comfortable with (seaborn, matplotlib).

**NOTE:** As we saw in the class distributions, there is about a 5:1 ratio for the target variable. Since the dataset is moderately imbalanced, I believe it'll be better to use a measure that is less sensitive to class imbalance than accuracy. As such, I will use ROC-AUC score.

In [24]:
# create list variable to store results
results_g2 = list()

## 7.1 RandomForestClassifier():

In [25]:
# random forests w/ 100 trees
clf_g2 = RandomForestClassifier(n_estimators=100, random_state=1859)

# use Cross Validation to run our model and generate the eval metrics
cv_g2 = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1859)
scores_g2 = cross_val_score(clf_g2, X_train_g2, y_train_g2, scoring='roc_auc', cv=cv_g2, n_jobs=-1) # using roc_auc since imbalanced dataset; more insightful than accuracy

# append scores to list, print mean and std
results_g2.append(scores_g2)
print(scores_g2.mean(), scores_g2.std())

0.9227947095980668 0.010400049284286322


## 7.2. BaggingClassifier():

In [26]:
# Bagging Classifier w/ 100 trees
clf_g2 = BaggingClassifier(n_estimators=100, random_state=1859)

# use Cross Validation to run our model and generate the eval metrics
cv_g2 = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1859)
scores_g2 = cross_val_score(clf_g2, X_train_g2, y_train_g2, scoring='roc_auc', cv=cv_g2, n_jobs=-1) # using roc_auc since imbalanced dataset; more insightful than accuracy

# append scores to list, print mean and std
results_g2.append(scores_g2)
print(scores_g2.mean(), scores_g2.std())

0.9225090980898315 0.00952896430748062


## 7.3. XGboostclassifier():

In [27]:
# XGBoost w/ 100 trees
clf_g2 = XGBClassifier(n_estimators=100, random_state=1859)

# use Cross Validation to run our model and generate the eval metrics
cv_g2 = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1859)
scores_g2 = cross_val_score(clf_g2, X_train_g2, y_train_g2, scoring='roc_auc', cv=cv_g2, n_jobs=-1) # using roc_auc since imbalanced dataset; more insightful than accuracy

# append scores to list, print mean and std
results_g2.append(scores_g2)
print(scores_g2.mean(), scores_g2.std())

0.9178714509108237 0.00939297343654051


## ROC-AUC Score Boxplot:

In [28]:
# define stacked classifier
def base_models():
  models_g2 = dict()
  models_g2["RF"] = RandomForestClassifier()
  models_g2["Bag"] = BaggingClassifier()
  models_g2["XGB"] = XGBClassifier()
  return models_g2

# evaluate the list of models
def eval_models(model):
  cv_g2 = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1859) # 5x3 cv
  scores_g2 = cross_val_score(model, X_train_g2, y_train_g2, scoring='roc_auc', cv=cv_g2, n_jobs=-1) # using roc_auc since imbalanced dataset; more insightful than accuracy
  return scores_g2

models_g2 = base_models()

# evaluate the models and store results
results_g2, names_g2 = list(), list()

for name, model in models_g2.items():
  scores = eval_models(model)
  results_g2.append(scores)
  names_g2.append(name)
  print('>%s %.3f (%.3f)' % (name, scores.mean(), scores.std()))

>RF 0.922 (0.011)
>Bag 0.896 (0.012)
>XGB 0.918 (0.009)


In [29]:
# store results in a df for future use
classmod_g2 = pd.DataFrame(np.transpose(results_g2), columns = ['RF', 'Bagging', 'XGB'])
classmod_g2 = pd.melt(classmod_g2.reset_index(), id_vars='index',value_vars=['RF', 'Bagging', 'XGB'])

# create boxplot to compare the performance of all models in order to select the best classifier
fig_g2 = px.box(classmod_g2, x="variable", y="value",color="variable",points='all',
labels={"variable": "Machine Learning Model",
        "value": "ROC-AUC Score"
        },title="Model Performance")

file_path = '/content/drive/MyDrive/Colab Notebooks/hw3/boxplot1.html'
pyo.plot(fig_g2, filename=file_path, auto_open=False)

fig_g2.show();

Based on the boxplots, Random Forests and XGBoost both appear to perform relatively similarly, followed by Bagging. However, based on the average and standard deviations in ROC-AUC score, Random Forests seem to slightly outperform XGBoost, so we'll proceed with Random Forests.

# 8. Classification with GridSearchCV (8 Points):

In [30]:
# define best model
model_g2 = RandomForestClassifier()

# define search space
space_g2 = dict()
space_g2['n_estimators'] = [100, 150, 200]
space_g2['max_depth'] = [None, 5, 10]
space_g2['min_samples_split'] = [2, 4, 8]
space_g2['min_samples_leaf'] = [2, 4, 8]

# define CV evaluation
cv_g2 = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1859) # reduced to 5x3 b/c taking WAAAAAY too long

# define grid search
search_g2 = GridSearchCV(model_g2, space_g2, scoring='roc_auc', n_jobs=-1, cv=cv_g2)

# fit grid search
result_g2 = search_g2.fit(X_train_g2, y_train_g2.ravel())

# summarize results
print('Best Score: %s' % result_g2.best_score_)
print('Best Hyperparameters: %s' % result_g2.best_params_)

Best Score: 0.9290677768857134
Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 200}


In [31]:
# get mean and standard deviation of the scores
mean_scores = result_g2.cv_results_['mean_test_score']
std_scores = result_g2.cv_results_['std_test_score']

# find the index of best hyperparams
best_index = np.argmax(mean_scores)

print('Best Mean Score: %s' % mean_scores[best_index])
print('Standard Deviation of Best Score: %s' % std_scores[best_index])

Best Mean Score: 0.9290677768857134
Standard Deviation of Best Score: 0.006745962484377069


# 9. Classification with RandomSearchCV (8 Points):

In [32]:
# define random search
search_g2 = RandomizedSearchCV(model_g2, space_g2, scoring='roc_auc', n_jobs=-1, cv=cv_g2, random_state=1859)

# fit random search
result_g2 = search_g2.fit(X_train_g2, y_train_g2.ravel())

# summarize result
print('Best Score: %s' % result_g2.best_score_)
print('Best Hyperparameters: %s' % result_g2.best_params_)

Best Score: 0.9286266319809725
Best Hyperparameters: {'n_estimators': 200, 'min_samples_split': 8, 'min_samples_leaf': 8, 'max_depth': 10}


In [33]:
# get mean and standard deviation of the scores
mean_scores = result_g2.cv_results_['mean_test_score']
std_scores = result_g2.cv_results_['std_test_score']

# find the index of best hyperparams
best_index = np.argmax(mean_scores)

print('Best Mean Score: %s' % mean_scores[best_index])
print('Standard Deviation of Best Score: %s' % std_scores[best_index])

Best Mean Score: 0.9286266319809725
Standard Deviation of Best Score: 0.006980590340529654


# 10. Comparison and Analysis (5 Points):

Compare the results from Q2, Q3, and Q4. Describe the best hyperparameters for all three experiments.

Our best performing model among Random Forests, XGBoost, and Bagging was the Random Forest Classifier.

The initial Random Forest Classifier simply had `n_estimators=100`, netting an ROC-AUC score of...

Using the hyperparameters from GridSearchCV of..., the model resulted in an ROC-AUC score of...

Lastly, the hyperparameters from RandomSearchCV of... resulted in an ROC-AUC score of...

As a result, the best set of hyperparameters was...